### Must install Lightning

In [1]:
!pip3 install lightning-bolts --quiet
!pip3 install torchmetrics
!pip3 install pytorch-lightning --quiet

!pip install kaggle

     |████████████████████████████████| 316 kB 5.3 MB/s 
     |████████████████████████████████| 584 kB 41.5 MB/s 
     |████████████████████████████████| 418 kB 59.2 MB/s 
     |████████████████████████████████| 140 kB 66.6 MB/s 
     |████████████████████████████████| 596 kB 23.3 MB/s 
     |████████████████████████████████| 1.1 MB 57.4 MB/s 
     |████████████████████████████████| 144 kB 16.6 MB/s 
     |████████████████████████████████| 94 kB 2.5 MB/s 
     |████████████████████████████████| 271 kB 55.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from torchvision import transforms
from pl_bolts.transforms.dataset_normalizations import cifar10_normalization
from torchvision.models.resnet import resnet18
import pytorch_lightning as pl
from pytorch_lightning import Trainer, LightningModule
import torch.nn as nn
import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10

In [3]:
from torchmetrics.functional import accuracy
from pytorch_lightning.callbacks import Callback
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint

In [4]:
EPOCHS = 200
LR = 0.1
MOMENTUM = 0.9
WEIGHT_DECAY = 5e-4
PRINT_FREQ = 50
TRAIN_BATCH=128
VAL_BATCH=128

In [5]:
GPU = 0

In [6]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 70 bytes


In [7]:
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d mengcius/cinic10

 98% 741M/754M [00:05<00:00, 229MB/s]
100% 754M/754M [00:05<00:00, 152MB/s]


In [8]:
!unzip cinic10.zip -d ./data/

Streaming output truncated to the last 5000 lines.
  inflating: ./data/valid/truck/n03632852_1173.png  
  inflating: ./data/valid/truck/n03632852_11766.png  
  inflating: ./data/valid/truck/n03632852_11887.png  
  inflating: ./data/valid/truck/n03632852_1191.png  
  inflating: ./data/valid/truck/n03632852_1192.png  
  inflating: ./data/valid/truck/n03632852_1195.png  
  inflating: ./data/valid/truck/n03632852_1207.png  
  inflating: ./data/valid/truck/n03632852_12128.png  
  inflating: ./data/valid/truck/n03632852_1214.png  
  inflating: ./data/valid/truck/n03632852_1237.png  
  inflating: ./data/valid/truck/n03632852_1248.png  
  inflating: ./data/valid/truck/n03632852_1261.png  
  inflating: ./data/valid/truck/n03632852_1264.png  
  inflating: ./data/valid/truck/n03632852_1292.png  
  inflating: ./data/valid/truck/n03632852_1313.png  
  inflating: ./data/valid/truck/n03632852_1319.png  
  inflating: ./data/valid/truck/n03632852_13311.png  
  inflating: ./data/valid/truck/n03632852_13

### fill in the transform statements below

In [9]:
imagenet_mean_RGB = [0.47889522, 0.47227842, 0.43047404]
imagenet_std_RGB = [0.229, 0.224, 0.225]
cinic_mean_RGB = [0.47889522, 0.47227842, 0.43047404]
cinic_std_RGB = [0.24205776, 0.23828046, 0.25874835]
cifar_mean_RGB = [0.4914, 0.4822, 0.4465]
cifar_std_RGB = [0.2023, 0.1994, 0.2010]

In [10]:
class CIFAR10DataModule(pl.LightningDataModule):
    def __init__(self, train_batch_size, val_batch_size, data_dir: str = './'):
        super().__init__()
        self.data_dir = data_dir
        self.train_batch_size = train_batch_size
        self.val_batch_size = val_batch_size
        
        self.transform_train = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(cifar_mean_RGB, cifar_std_RGB),
        ])
        self.transform_val = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(cifar_mean_RGB, cifar_std_RGB),
        ])
        
        self.dims = (3, 32, 32)
        self.num_classes = 10

    def prepare_data(self):
        # download 
        CIFAR10(self.data_dir, train=True, download=True)
        CIFAR10(self.data_dir, train=False, download=True)

    def setup(self, stage=None):
        # Assign train/val datasets for use in dataloaders
        if stage == 'fit' or stage is None:
#            cifar_full = CIFAR10(self.data_dir, train=True, transform=self.transform)
#            self.cifar_train, self.cifar_val = random_split(cifar_full, [45000, 5000])
            self.cifar_train = CIFAR10(self.data_dir, train=True, transform=self.transform_train)
            self.cifar_val = CIFAR10(self.data_dir, train=False, transform=self.transform_val)

        # Assign test dataset for use in dataloader(s)
        if stage == 'test' or stage is None:
            self.cifar_test = CIFAR10(self.data_dir, train=False, transform=self.transform_val)

    def train_dataloader(self):
        return DataLoader(self.cifar_train, batch_size=self.train_batch_size, num_workers = 2, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.cifar_val, batch_size=self.val_batch_size, num_workers = 2)

    def test_dataloader(self):
        return DataLoader(self.cifar_test, batch_size=self.batch_size, num_workers = 2)

In [11]:
dm = CIFAR10DataModule(TRAIN_BATCH, VAL_BATCH)
dm.prepare_data()
dm.setup()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/core/datamodule.py:149: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./cifar-10-python.tar.gz to ./
Files already downloaded and verified


In [12]:
MODEL_CKPT_PATH = 'model/'
MODEL_CKPT = 'model/model-{epoch:02d}-{val_loss:.2f}'

checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    filename=MODEL_CKPT ,
    save_top_k=3,
    mode='min')

In [13]:
# Samples required by the custom ImagePredictionLogger callback to log image predictions.
val_samples = next(iter(dm.val_dataloader()))
val_imgs, val_labels = val_samples[0], val_samples[1]
val_imgs.shape, val_labels.shape

(torch.Size([128, 3, 32, 32]), torch.Size([128]))

In [14]:
early_stop_callback = EarlyStopping(
   monitor='val_loss',
   patience=3,
   verbose=False,
   mode='min'
)

### Complete the training, validation, and optimizer methods below

In [15]:
class LitResnet18(LightningModule):
    def __init__(self, learning_rate, momentum, weight_decay):
        super().__init__()
        self.nn = resnet18(pretrained = False, progress  = True)
        self.nn.fc = nn.Linear(self.nn.fc.in_features, 10)
        self.lr = learning_rate
        self.momentum = momentum
        self.weight_decay = weight_decay
        self.criterion = nn.CrossEntropyLoss().cuda(GPU)
    
    def forward(self, x):
        return self.nn.forward(x)
    
    def training_step(self, batch, batch_idx):
        x,y = batch
        logits = self.nn(x)
        loss = self.criterion(logits, y)
        # training metrics
        preds = torch.argmax(logits, dim=1)
        acc = accuracy(preds, y)
        self.log('train_loss', loss, on_step=True, on_epoch=True, logger=False)
        self.log('train_acc', acc, on_step=True, on_epoch=True, logger=False)
        if batch_idx % PRINT_FREQ == 0:
          print("train step! " + str(batch_idx) + " train loss: " + str(loss.item()) + " train acc " + str(acc.item()))        
        return loss     
        
        
    def validation_step(self, batch, batch_idx):
        x,y = batch
        logits = self.nn(x)
        loss = self.criterion(logits, y) 
        # validation metrics
        preds = torch.argmax(logits, dim=1)
        acc = accuracy(preds, y)
        self.log('val_loss', loss, prog_bar=True)
        self.log('val_acc', acc, prog_bar=True)
        if batch_idx % PRINT_FREQ == 0:
          print("val step! " + str(batch_idx) + " val loss: " + str(loss.item()) + " val acc " + str(acc.item()))
        return loss  
        
        
        
    def configure_optimizers(self):
        optimizer = torch.optim.SGD(model.parameters(), self.lr, momentum=self.momentum, weight_decay=self.weight_decay)
        return optimizer

In [16]:
# model = resnet18(pretrained = False, progress  = True)
model = LitResnet18(LR, MOMENTUM, WEIGHT_DECAY)


In [17]:
# Initialize a trainer
trainer = pl.Trainer(max_epochs=EPOCHS,
                     progress_bar_refresh_rate=20, 
                     gpus=1, 
                     logger=None,
                     callbacks=[early_stop_callback],
                     checkpoint_callback=checkpoint_callback)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:152: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=<pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint object at 0x7fd211287090>)` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_checkpointing=<pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint object at 0x7fd211287090>)`.
  f"Setting `Trainer(checkpoint_callback={checkpoint_callback})` is deprecated in v1.5 and will "
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:97: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=20)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Tr

In [18]:
trainer.fit(model, dm)

Files already downloaded and verified
Files already downloaded and verified


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params
-----------------------------------------------
0 | nn        | ResNet           | 11.2 M
1 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.727    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

val step! 0 val loss: 2.3216991424560547 val acc 0.1484375


/usr/local/lib/python3.7/dist-packages/torchmetrics/utilities/prints.py:36: UserWarning: Torchmetrics v0.9 introduced a new argument class property called `full_state_update` that has
                not been set for this class (_ResultMetric). The property determines if `update` by
                default needs access to the full metric state. If this is not the case, significant speedups can be
                achieved and we recommend setting this to `False`.
                We provide an checking function
                `from torchmetrics.utilities import check_forward_no_full_state`
                that can be used to check if the `full_state_update=True` (old and potential slower behaviour,
                default for now) or if `full_state_update=False` can be used safely.
                
  warnings.warn(*args, **kwargs)


...

Validation: 0it [00:00, ?it/s]

val step! 0 val loss: 0.7187463641166687 val acc 0.734375
val step! 50 val loss: 0.792082667350769 val acc 0.765625
train step! 0 train loss: 0.38063058257102966 train acc 0.8671875
train step! 50 train loss: 0.49629607796669006 train acc 0.8046875
train step! 100 train loss: 0.8459348678588867 train acc 0.7265625
train step! 150 train loss: 0.47931644320487976 train acc 0.8125
train step! 200 train loss: 0.721611499786377 train acc 0.7421875
train step! 250 train loss: 0.7879101037979126 train acc 0.734375
train step! 300 train loss: 0.38726818561553955 train acc 0.859375
train step! 350 train loss: 0.8008633852005005 train acc 0.703125


Validation: 0it [00:00, ?it/s]

val step! 0 val loss: 0.7680980563163757 val acc 0.7421875
val step! 50 val loss: 0.7471539378166199 val acc 0.7265625
train step! 0 train loss: 0.4366177022457123 train acc 0.8515625
train step! 50 train loss: 0.48352888226509094 train acc 0.8515625
train step! 100 train loss: 0.663245677947998 train acc 0.7890625
train step! 150 train loss: 0.5197433233261108 train acc 0.8125
train step! 200 train loss: 0.6245307326316833 train acc 0.8125
train step! 250 train loss: 0.6342105865478516 train acc 0.78125
train step! 300 train loss: 0.6228864789009094 train acc 0.78125
train step! 350 train loss: 0.6454299092292786 train acc 0.7578125


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:724: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
